<font color=red>测试初始化<font>

In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist']


INFO 2025-02-07 02:36:49,165 storage 使用 COSStorage 初始化存储
INFO 2025-02-07 02:36:49,166 storage 初始化 COSStorage
INFO 2025-02-07 02:36:49,166 storage Storage backend initialization with args: (), kwargs: {}
INFO 2025-02-07 02:36:49,166 storage ✅ 成功初始化 COSStorage
INFO 2025-02-07 02:36:49,167 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist


In [2]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project, ProjectHistory
from apps.files.models import FileRecord, FileProjectLink

In [3]:
# 准备测试所需的 user, project, file_record  (其中project与file_record关联)
User = get_user_model()

# 获取已存在的测试数据

# 获取已存在的用户
user = User.objects.get(phone='18501771516')
print(f"用户: {user.phone}")
        
# 获取已存在的项目
project = Project.objects.get(project_name='测试项目1')
print(f"项目: {project.project_name}")
        
# 获取已存在的文件
file_record = FileRecord.objects.get(id='2')
print(f"文件: {file_record.name}")

# 5. 使用 select_related 优化查询 项目关联的文件
optimized_files = project.file_links.select_related('file_record').all()
print("\n优化查询后的文件：")
for link in optimized_files:
    print(f"- {link.file_record.name}")


用户: 18501771516
项目: 测试项目1
文件: Test 文本.docx

优化查询后的文件：
- Test 文本.docx


In [4]:
# 检查项目与文件的关联
print("项目的所有关联文件：")
for file_link in project.file_links.all():
    print(f"- {file_link.file_record.name} ({file_link.link_type})")


项目的所有关联文件：
- Test 文本.docx (ATTACHMENT)


<font color=red>1. 测试分析模型 Models.py：创建和状态自动更新<font>

In [ ]:
# 创建文档分析
analyses = DocumentAnalysis.objects.filter(project=project, file_record=file_record)
isAnalysisExist = analyses.exists()
if isAnalysisExist:
    print(f"分析已存在，跳过创建")
    this_analysis = analyses.filter(title="测试分析")
    this_analysis.update(status=DocumentAnalysis.AnalysisStatus.PENDING)
    print(f"初始化文档分析为PENDING状态: {this_analysis.first().status}")

else:
    analysis = DocumentAnalysis.objects.create(
        title="测试分析",
        project=project,
        file_record=file_record,
        created_by=user
    )


In [ ]:
# 打印分析列表
analyses = DocumentAnalysis.objects.filter(title__startswith='测试分析')
for analysis in analyses:
    print(f"分析号:{analysis.id}\n",
          f"分析名称：{analysis.title}\n",
          f"分析所在项目：{analysis.project.project_name}\n", 
          f"分析的文件：{analysis.file_record.name}\n" ,
          f"分析的阶段：{analysis.status}\n",
          f"分析的问题：{analysis.analysis_questions}\n",
          f"分析结果：{analysis.analysis_result}\n",
          f"分析创建者：{analysis.created_by.phone}\n",
          f"分析用时：{analysis.processing_time}\n",
          )

In [ ]:
# 测试状态转换流程
if analysis.status == DocumentAnalysis.AnalysisStatus.PENDING:
    print("1.可以测试文档分析从PENDING到PROCESSING的流转：")
    analysis.start_analysis()
    print(f"开始分析后状态: {analysis.status}")
else:
    print(f"1. 测试文档分析状态在{analysis.status}，不能使用start_analysis()方法")


In [ ]:
# 模拟分析结果
sample_result = [
    {
        "question": "资质要求",
        "answer": "需要具备建筑施工总承包特级资质"
    },
    {
        "question": "技术参数",
        "answer": "项目规模：建筑面积50000平方米"
    }
]
# 完成分析
if analysis.status == DocumentAnalysis.AnalysisStatus.PROCESSING:
    print("2.可以测试文档分析从PROCESSING到COMPLETED的流转：")
    analysis.complete_analysis(result=sample_result)
    print(f"完成分析后状态: {analysis.status}")

else:
    print(f"2. 测试文档分析状态在{analysis.status}，不能使用complete_analysis()方法")


In [ ]:
# 确认分析结果
confirmed_results = [
    {
        "question": "资质要求",
        "answer": "需要具备建筑施工总承包特级资质",
        "comment": "确认无误"
    },
    {
        "question": "技术参数",
        "answer": "项目规模：建筑面积50000平方米",
        "comment": "数据已核实"
    }
]

# 完成分析
if analysis.status == DocumentAnalysis.AnalysisStatus.COMPLETED:
    print("3.可以测试文档分析从COMPLETED到CONFIRMED的流转：")
    analysis.confirm_analysis(user=user, confirmed_results=confirmed_results)
    print(f"完成分析后状态: {analysis.status}")

else:
    print(f"3. 测试文档分析状态在{analysis.status}，不能使用confirm_analysis()方法")


In [ ]:
# 测试错误状态转换
print("\n2. 测试错误状态转换处理：")
try:
    # 创建新的分析实例用于测试失败场景
    failed_analysis = DocumentAnalysis.objects.create(
        title="测试失败分析",
        project=project,
        file_record=file_record,
        created_by=user
    )
    print(f"创建失败分析的状态: {failed_analysis.status}")
    # 直接尝试确认一个未完成的分析
    failed_analysis.confirm_analysis(user=user, confirmed_results=[])
except InvalidStatusTransition as e:
    print(f"预期的错误捕获: {str(e)}")

In [ ]:
# failed_analysis.delete()

In [ ]:
# 测试失败流程
failed_analysis.start_analysis()
failed_analysis.fail_analysis(error_message="文档格式不支持")
print(f"失败分析状态: {failed_analysis.status}")
print(f"错误信息: {failed_analysis.error_message}")

In [ ]:
# 查看分析结果
print("\n3. 查看最终分析结果：")
final_analysis = DocumentAnalysis.objects.get(id=analysis.id)
print(f"分析标题: {final_analysis.title}")
print(f"当前状态: {final_analysis.status}")
print(f"分析结果: {final_analysis.analysis_result}")
print(f"确认时间: {final_analysis.confirmed_at}")
print(f"确认用户: {final_analysis.confirmed_by.phone}")

<font color=red>2.Serializers.py测试<font>

In [5]:
from apps.doc_analysis.serializers import ( 
    DocumentAnalysisBaseSerializer, 
    DocumentAnalysisCreateSerializer,
    AnalysisResultUpdateSerializer,
    AnalysisConfirmationSerializer,
    DocumentAnalysisDisplaySerializer
)

In [21]:
# 准备测试数据
test_file = FileRecord.objects.get(id='2')
test_project = Project.objects.get(project_name='测试项目1')
test_user = User.objects.get(phone='18501771516')

In [22]:
# 模拟请求类
class MockRequest:
    def __init__(self, user=None):
        self.user = test_user
        self.method = 'POST'  # 可以根据需要设置请求方法
        self.META = {}        # 请求元数据
        self.session = {}     # 会话数据

# 创建模拟请求实例
mock_request = MockRequest(user=test_user)


In [23]:
# 1. 测试创建序列化器
print("=== 测试创建序列化器 ===")
create_data = {
    "project_id": test_project.id,
    "file_record_id": test_file.id,
    "title": "序列化器测试分析",
    "analysis_questions": ["资质要求", "技术参数"]
}

create_serializer = DocumentAnalysisCreateSerializer(
    data=create_data,
    context={'request': MockRequest()}
)

if create_serializer.is_valid():
    new_analysis = create_serializer.save()
    print(f"✅ 创建成功 - ID: {new_analysis.id}")
else:
    print(f"❌ 创建失败 - 错误: {create_serializer.errors}")

=== 测试创建序列化器 ===
✅ 创建成功 - ID: 6


In [25]:
# 打印new_analysis
for analysis in [new_analysis]:
    print(f"分析号:{analysis.id}\n",
          f"分析名称：{analysis.title}\n",
          f"分析所在项目：{analysis.project.project_name}\n", 
          f"分析的文件：{analysis.file_record.name}\n" ,
          f"分析的阶段：{analysis.status}\n",
          f"分析的问题：{analysis.analysis_questions}\n",
          f"分析结果：{analysis.analysis_result}\n",
          f"分析创建者：{analysis.created_by.phone}\n",
          f"分析用时：{analysis.processing_time}\n",
          )

分析号:6
 分析名称：序列化器测试分析
 分析所在项目：测试项目1
 分析的文件：Test 文本.docx
 分析的阶段：PENDING
 分析的问题：['资质要求', '技术参数']
 分析结果：None
 分析创建者：18501771516
 分析用时：None



In [11]:
# 测试无效文件类型
print("\n测试无效文件类型:")
invalid_file = FileRecord.objects.create(
    name="test.txt",
    type="TXT",
    size=1024,
    owner=test_user,  # 添加必需的owner字段
    version=1,        # 添加必需的version字段
    processing_status='NONE',  # 添加必需的processing_status字段
    created_by=test_user.phone  # 添加必需的created_by字段
)

invalid_data = create_data.copy()
invalid_data["file_record_id"] = invalid_file.id

invalid_serializer = DocumentAnalysisCreateSerializer(
    data=invalid_data,
    context={'request': mock_request}
)

if not invalid_serializer.is_valid():
    print(f"✅ 正确捕获错误: {invalid_serializer.errors}")


测试无效文件类型:
✅ 正确捕获错误: {'file_record_id': [ErrorDetail(string='目前仅支持DOCX格式文件', code='invalid')]}


In [13]:
# 2. 测试结果更新序列化器
print("\n=== 测试结果更新序列化器 ===")
update_data = {

            "question": "资质要求",
            "answer": "需要具备建筑施工总承包特级资质",
            "context": ["上下文段落1", "上下文段落2"],
            "confidence": 0.95
        }

result_serializer = AnalysisResultUpdateSerializer(
    instance=new_analysis,
    data=update_data,
    context={'request': mock_request}
)

if result_serializer.is_valid():
    updated = result_serializer.save()
    print(f"✅ 结果更新成功 - 最新结果: {updated.analysis_result[-1]}")
else:
    print(f"❌ 更新失败 - 错误: {result_serializer.errors}")


=== 测试结果更新序列化器 ===
❌ 更新失败 - 错误: {'non_field_errors': [ErrorDetail(string='当前状态不允许更新分析结果', code='invalid')]}


In [26]:
# 3. 测试确认序列化器
print("\n=== 测试确认序列化器 ===")
confirmation_data = {
            "question": "资质要求",
            "answer": "需要具备建筑施工总承包特级资质",
            "comment": "测试确认"
        }


confirmation_serializer = AnalysisConfirmationSerializer(
    instance=new_analysis,
    data=confirmation_data,
    context={'request': mock_request}
)

if confirmation_serializer.is_valid():
    confirmed = confirmation_serializer.save()
    print(f"✅ 确认成功 - 状态: {confirmed.status}")
    print(f"确认信息: {confirmed.analysis_result[0].get('confirmation')}")
else:
    print(f"❌ 确认失败 - 错误: {confirmation_serializer.errors}")


=== 测试确认序列化器 ===
❌ 确认失败 - 错误: {'confirmed_answers': [ErrorDetail(string='该字段是必填项。', code='required')]}
